In [81]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
import os

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

from psycopg2.extensions import register_adapter, AsIs

def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)

def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

register_adapter(np.float64, addapt_numpy_float64)
register_adapter(np.int64, addapt_numpy_int64)

In [82]:
from config import yelp_api_key, darksky_api_key, PGHOST, PGDATABASE, PGUSER, PGPASSWORD
from restaurant_info import restaurantLocation
from weather import Weather

### Get Latitude & Longitude from Yelp API

In [83]:
search_business = 'The Counting Room' # Not the actual Restaurant 
location = 'Brooklyn, NY'

In [84]:
# Make Yelp API Call to get Latitude & Longitude for Business
rest_loc = restaurantLocation(search_business, location)
lat, long = rest_loc.get_lat_long()

Weather Location: The Counting Room


### Import / Clean / Prep File

In [85]:
w_start = '2017-01-01'
w_end = '2019-06-30'

# Restaurant File
sales_file = 'csv/new_through_06_09_separate_outside.csv'

# Weather File
weather_csv_file = f'csv/weather_{w_start}_to_{w_end}.csv'

In [86]:
 # Read in Sales File
data = pd.read_csv(sales_file, index_col = 'date', parse_dates=True)
df = pd.DataFrame(data)

In [87]:
df.head()

,inside_sales,outside_sales,inside_covers,outside_covers,reserved_covers,walkin_covers,waitlist_covers,no_show_covers,no_show_parties,no_show_cover_rate
date,,,,,,,,,,
2017-01-02,13159.84,0.0,174.0,0,106.0,26.0,42.0,17.0,6.0,0.160377
2017-01-03,12442.11,0.0,181.0,0,119.0,31.0,31.0,14.0,4.0,0.117647
2017-01-04,12927.64,0.0,174.0,0,131.0,17.0,26.0,5.0,2.0,0.038168
2017-01-05,14457.79,0.0,191.0,0,138.0,25.0,28.0,4.0,2.0,0.028986
2017-01-06,15331.97,0.0,200.0,0,130.0,16.0,54.0,6.0,3.0,0.046154


In [88]:
df.tail()

,inside_sales,outside_sales,inside_covers,outside_covers,reserved_covers,walkin_covers,waitlist_covers,no_show_covers,no_show_parties,no_show_cover_rate
date,,,,,,,,,,
2019-06-26,12605.00,4487.50,179.0,67,149.0,0.0,97.0,6.0,2.0,0.040268
2019-06-27,13675.62,5443.00,170.0,80,143.0,0.0,107.0,5.0,2.0,0.034965
2019-06-28,14171.25,6180.50,177.0,74,145.0,0.0,106.0,12.0,4.0,0.082759
2019-06-29,14695.50,5369.75,189.0,72,135.0,3.0,123.0,10.0,5.0,0.074074
2019-06-30,14073.24,4925.50,221.0,75,156.0,2.0,138.0,4.0,1.0,0.025641


In [89]:
def prep_df(df):
    
        # Fill NaN
        df.fillna(0, inplace=True)
        
        # Create ID Primary Key from Datetime
        df['id'] = df.index.strftime('%Y%m%d')
        
        return df
    
df = prep_df(df)

In [90]:
df.head()

,inside_sales,outside_sales,inside_covers,outside_covers,reserved_covers,walkin_covers,waitlist_covers,no_show_covers,no_show_parties,no_show_cover_rate,id
date,,,,,,,,,,,
2017-01-02,13159.84,0.0,174.0,0,106.0,26.0,42.0,17.0,6.0,0.160377,20170102
2017-01-03,12442.11,0.0,181.0,0,119.0,31.0,31.0,14.0,4.0,0.117647,20170103
2017-01-04,12927.64,0.0,174.0,0,131.0,17.0,26.0,5.0,2.0,0.038168,20170104
2017-01-05,14457.79,0.0,191.0,0,138.0,25.0,28.0,4.0,2.0,0.028986,20170105
2017-01-06,15331.97,0.0,200.0,0,130.0,16.0,54.0,6.0,3.0,0.046154,20170106


In [91]:
dfw = pd.read_csv(weather_csv_file, index_col='date', parse_dates=True)

In [92]:
dfw = prep_df(dfw)

### Connect Notebook to AWS Postgres Instance

In [78]:
# Connect to RDS-Postgres DB

def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

conn, cursor = connect()

Connected!


### Create Sales & Reservations Tables

In [80]:
create rev_center_table = """
    CREATE TABLE IF NOT EXISTS rev_center(
        id INTEGER PRIMARY KEY NOT NULL,
        name NOT NULL)
"""

create_inside_sales_table = """
    CREATE TABLE IF NOT EXISTS sales(
        id INTEGER PRIMARY KEY NOT NULL,
        date DATE,
        inside_sales NUMERIC (7, 2)
    )
    """

create_resy_table = """
    CREATE TABLE IF NOT EXISTS reservations(
        id INTEGER PRIMARY KEY NOT NULL,
        date DATE,
        inside_covers INTEGER,
        outside_covers INTEGER,
        reserved_covers INTEGER,
        walkin_covers INTEGER,
        waitlist_covers INTEGER,
        no_show_covers INTEGER,
        no_show_parties INTEGER
    )
    """

cursor.execute(create_sales_table)
cursor.execute(create_resy_table)
conn.commit()

In [67]:
cursor.execute("CREATE TABLE weather ( \
                DATE DATE, \
                date_id INT PRIMARY KEY, \
                apparent_temperature NUMERIC (4, 2), \
                humidity NUMERIC (3, 2), \
                precip_intensity_max NUMERIC (5, 4), \
                precip_max_time TIME, \
                precip_prob NUMERIC (3, 2), \
                precip_type TEXT, \
                pressure NUMERIC (6,2), \
                summary TEXT, \
                temperature NUMERIC (4, 2));")

conn.commit()

### Populate Sales & Reservation Databases

In [93]:
def populate_sales(df):
    
    for row in range(len(df)):
            
        cursor.execute("INSERT INTO sales (id, date, inside_sales, outside_sales) VALUES (%s, %s, %s, %s)", 
            (df.iloc[row]['id'],
            pd.to_datetime(df.index[row]),
            df.iloc[row]['inside_sales'],
            df.iloc[row]['outside_sales']) )
        conn.commit()
    
populate_sales(df)
                

In [94]:
def populate_reservations(df):
    
    for row in range(len(df)):
        
        cursor.execute("""
            INSERT INTO reservations (id, date, inside_covers, outside_covers, reserved_covers,
            walkin_covers, waitlist_covers, no_show_covers, no_show_parties) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                (df.iloc[row]['id'],
                pd.to_datetime(df.index[row]),
                df.iloc[row]['inside_covers'],
                df.iloc[row]['outside_covers'],
                df.iloc[row]['reserved_covers'],
                df.iloc[row]['walkin_covers'],
                df.iloc[row]['waitlist_covers'],
                df.iloc[row]['no_show_covers'],
                df.iloc[row]['no_show_parties']) )
        conn.commit()
        
populate_reservations(df)

In [96]:
def populate_weather(df):
    
    for row in range(len(df)):
        
        cursor.execute("""
            INSERT INTO weather (date, date_id, apparent_temperature, humidity, precip_intensity_max, precip_max_time,
            precip_prob, precip_type, pressure, summary, temperature) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                (pd.to_datetime(df.index[row]),
                 df.iloc[row]['id'],
                 df.iloc[row]['apparent_temperature'],
                 df.iloc[row]['humidity'],
                 df.iloc[row]['precip_intensity_max'],
                 df.iloc[row]['precip_max_time'],
                 df.iloc[row]['precip_prob'],
                 df.iloc[row]['precip_type'],
                 df.iloc[row]['pressure'],
                 df.iloc[row]['summary'],
                 df.iloc[row]['temperature']) )
        conn.commit()

populate_weather(dfw)
        

In [98]:
conn.close()

### 